###Import Library

In [1]:
!pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 9.2 MB/s eta 0:00:00


In [2]:
import time
import numpy as np
import pandas as pd
import gymnasium as gym
import matplotlib.pyplot as plt
from IPython import display

###Initialize with FrozenLake-v1

In [3]:
env = gym.make('FrozenLake-v1', render_mode="ansi")

In [4]:
env.P[0][3] # Transition model

/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.P to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.P` for environment variables or `env.get_wrapper_attr('P')` that will search the reminding wrappers.
  logger.warn(


[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [5]:
env.observation_space.n

16

In [6]:
env.action_space.n

4

In [7]:
def play(env, policy, render=False):
    state, _ = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info, _ = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            print(env.render())
            time.sleep(0.5)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [8]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0, True)


  (Left)
SFFF
FHFH
FFFH
HFFG



(0.0, 9)

In [9]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play(env, policy_1, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(0.0, 7)

In [10]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play(env, policy_2, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(0.0, 5)

In [11]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play(env, policy_3, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 56)

In [12]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')
    return success, np.mean(list_of_steps)

In [13]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play_multiple_times(env, policy_0, 1000)

Number of successes: 0/1000
Average number of steps: nan


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


(0, nan)

In [14]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play_multiple_times(env, policy_1, 1000)

Number of successes: 53/1000
Average number of steps: 11.150943396226415


(53, 11.150943396226415)

In [15]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play_multiple_times(env, policy_2, 1000)

Number of successes: 101/1000
Average number of steps: 16.722772277227723


(101, 16.722772277227723)

In [16]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play_multiple_times(env, policy_3, 1000)

Number of successes: 785/1000
Average number of steps: 43.44203821656051


(785, 43.44203821656051)

In [17]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9, verbose=True):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value

        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
          if verbose:
            print(f'Converged at {i}-th iteration.')
          break

    return v_values

In [18]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v_values_0 = policy_evaluation(env, policy_0)
print(v_values_0)

Converged at 0-th iteration.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [19]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
v_values_1 = policy_evaluation(env, policy_1)
print(v_values_1)

Converged at 48-th iteration.
[0.01904157 0.01519815 0.03161906 0.02371389 0.02538879 0.
 0.06648515 0.         0.05924054 0.13822794 0.18999823 0.
 0.         0.21152109 0.56684236 0.        ]


In [20]:
np.all(v_values_1 >= v_values_0)

True

In [21]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
v_values_2 = policy_evaluation(env, policy_2)
print(v_values_2)

Converged at 53-th iteration.
[0.02889625 0.01951972 0.03616977 0.0271268  0.04790519 0.
 0.07391985 0.         0.08288277 0.19339319 0.21022995 0.
 0.         0.35153135 0.62684674 0.        ]


In [22]:
np.all(v_values_2 >= v_values_1)

True

In [23]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
v_values_3 = policy_evaluation(env, policy_3)
print(v_values_3)

Converged at 80-th iteration.
[0.06888666 0.06141097 0.07440714 0.05580443 0.09185068 0.
 0.11220679 0.         0.14543323 0.24749485 0.29961611 0.
 0.         0.37993438 0.63901935 0.        ]


In [24]:
np.all(v_values_3 >= v_values_2)

True

In [25]:
def value_iteration(env, max_iters=500, gamma=0.9, verbose=True):
    # initialize
    v_values = np.zeros(env.observation_space.n)
    temp = 0
    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []

            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])

                q_values.append(q_value)

            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]

        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
          if verbose:
            print(f'Converged at {i}-th iteration.')
            temp = i
          break

    return v_values, temp

In [26]:
optimal_v_values, value_iter = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 79-th iteration.


In [27]:
optimal_v_values

array([0.06888615, 0.06141054, 0.07440682, 0.05580409, 0.09185022,
       0.        , 0.11220663, 0.        , 0.14543286, 0.2474946 ,
       0.29961593, 0.        , 0.        , 0.3799342 , 0.63901926,
       0.        ])

In [28]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int32)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])

            q_values.append(q_value)

        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action

    return policy

In [29]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

In [30]:
optimal_policy

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0], dtype=int32)

In [31]:
play(env, optimal_policy, True)

  (Left)
SFFF
FHFH
FFFH
HFFG



(0.0, 28)

In [32]:
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 788/1000
Average number of steps: 42.55964467005076


(788, 42.55964467005076)

In [33]:
def policy_iteration(env, max_iters=500, gamma=0.9, verbose=True):
    policy = np.zeros(env.observation_space.n, dtype=np.int32)
    temp = 0
    for i in range(max_iters):
        v_values = policy_evaluation(env, policy, 500, 0.9, False)
        new_policy = policy_extraction(env, v_values, 0.9)
        if np.array_equal(new_policy, policy):
          if verbose:
            print(f'Converged at {i}-th iteration.')
            temp = i
          break
        policy = new_policy

    return policy, temp


In [34]:
play_multiple_times(env, policy_iteration(env, 500, 0.9)[0], 1000)

Converged at 5-th iteration.
Number of successes: 779/1000
Average number of steps: 41.397946084724005


(779, 41.397946084724005)

###Experiment


In [35]:
MAX_ITERS = 2000
MAX_EPISODES = 10000
GAMMA = 0.9

In [36]:
statistics = []

FrozenLake-v1

In [37]:
env = gym.make('FrozenLake-v1')

In [38]:
vi_iter_avg = 0
pi_iter_avg = 0

vi_rate_avg = 0
pi_rate_avg = 0

vi_step_avg = 0
pi_step_avg = 0

vi_time_avg = 0
pi_time_avg = 0

vi_find_optimal_avg = 0
pi_find_optimal_avg = 0

for i in range(0, 3):
    print('Iter ', i + 1)
    start_vi = time.time()
    vi_value,vi_iter = value_iteration(env, MAX_ITERS, GAMMA)
    vi_time = time.time() - start_vi
    policy_from_value = policy_extraction(env, vi_value, GAMMA)
    start_vi_avg = time.time()
    vi_successes, avg_vi_steps = play_multiple_times(env, policy_from_value, MAX_EPISODES)
    vi_avg = time.time() - start_vi_avg
    start_pi = time.time()
    pi, pi_iter = policy_iteration(env, MAX_ITERS, gamma=GAMMA)
    pi_time = time.time() - start_pi
    start_pi_avg = time.time()
    pi_successes, avg_pi_steps = play_multiple_times(env, pi, MAX_EPISODES)
    pi_avg = time.time() - start_pi_avg
    print()
    print(f'Number of successes of Value Iteration in FrozenLake-v1 : {vi_successes}/{MAX_EPISODES},  Time find the optimal policy : {vi_time}s')
    print(f'Number of successes of Policy Iteration in FrozenLake-v1 : {pi_successes}/{MAX_EPISODES}, Time find the optimal policy : {pi_time}s')
    print()
    print('-' * 50)
    print()
    vi_iter_avg += vi_iter/3
    pi_iter_avg += pi_iter/3

    vi_rate_avg += float(vi_successes)/MAX_EPISODES/3
    pi_rate_avg += float(pi_successes)/MAX_EPISODES/3

    vi_step_avg += avg_vi_steps/3
    pi_step_avg += avg_pi_steps/3

    vi_time_avg += vi_avg/MAX_EPISODES/3
    pi_time_avg += pi_avg/MAX_EPISODES/3

    vi_find_optimal_avg += vi_time/3
    pi_find_optimal_avg += pi_time/3

statistics.append(['FrozenLake-v1', vi_iter_avg, pi_iter_avg, vi_rate_avg, pi_rate_avg, vi_step_avg, pi_step_avg, vi_time_avg, pi_time_avg, vi_find_optimal_avg, pi_find_optimal_avg])

Iter  1
Converged at 79-th iteration.
Number of successes: 7810/10000
Average number of steps: 43.78911651728553
Converged at 5-th iteration.
Number of successes: 7763/10000
Average number of steps: 42.920133968826484

Number of successes of Value Iteration in FrozenLake-v1 : 7810/10000,  Time find the optimal policy : 0.10305333137512207s
Number of successes of Policy Iteration in FrozenLake-v1 : 7763/10000, Time find the optimal policy : 0.15805315971374512s

--------------------------------------------------

Iter  2
Converged at 79-th iteration.
Number of successes: 7821/10000
Average number of steps: 43.02314282061118
Converged at 5-th iteration.
Number of successes: 7796/10000
Average number of steps: 43.78373524884556

Number of successes of Value Iteration in FrozenLake-v1 : 7821/10000,  Time find the optimal policy : 0.06744980812072754s
Number of successes of Policy Iteration in FrozenLake-v1 : 7796/10000, Time find the optimal policy : 0.10482621192932129s

-----------------

FrozenLake8x8-v1

In [39]:
env = gym.make('FrozenLake8x8-v1')

In [40]:
vi_iter_avg = 0
pi_iter_avg = 0

vi_rate_avg = 0
pi_rate_avg = 0

vi_step_avg = 0
pi_step_avg = 0

vi_time_avg = 0
pi_time_avg = 0

vi_find_optimal_avg = 0
pi_find_optimal_avg = 0

for i in range(0, 3):
    print('Iter ', i + 1)
    start_vi = time.time()
    vi_value,vi_iter = value_iteration(env, MAX_ITERS, GAMMA)
    vi_time = time.time() - start_vi
    policy_from_value = policy_extraction(env, vi_value, GAMMA)
    start_vi_avg = time.time()
    vi_successes, avg_vi_steps = play_multiple_times(env, policy_from_value, MAX_EPISODES)
    vi_avg = time.time() - start_vi_avg
    start_pi = time.time()
    pi, pi_iter = policy_iteration(env, MAX_ITERS, gamma=GAMMA)
    pi_time = time.time() - start_pi
    start_pi_avg = time.time()
    pi_successes, avg_pi_steps = play_multiple_times(env, pi, MAX_EPISODES)
    pi_avg = time.time() - start_pi_avg
    print()
    print(f'Number of successes of Value Iteration in FrozenLake8x8-v1 : {vi_successes}/{MAX_EPISODES},  Time find the optimal policy : {vi_time}s')
    print(f'Number of successes of Policy Iteration in FrozenLake8x8-v1 : {pi_successes}/{MAX_EPISODES}, Time find the optimal policy : {pi_time}s')
    print()
    print('-' * 50)
    print()
    vi_iter_avg += vi_iter/3
    pi_iter_avg += pi_iter/3

    vi_rate_avg += float(vi_successes)/MAX_EPISODES/3
    pi_rate_avg += float(pi_successes)/MAX_EPISODES/3

    vi_step_avg += avg_vi_steps/3
    pi_step_avg += avg_pi_steps/3

    vi_time_avg += vi_avg/MAX_EPISODES/3
    pi_time_avg += pi_avg/MAX_EPISODES/3

    vi_find_optimal_avg += vi_time/3
    pi_find_optimal_avg += pi_time/3

statistics.append(['FrozenLake8x8-v1', vi_iter_avg, pi_iter_avg, vi_rate_avg, pi_rate_avg, vi_step_avg, pi_step_avg, vi_time_avg, pi_time_avg, vi_find_optimal_avg, pi_find_optimal_avg])

Iter  1
Converged at 117-th iteration.
Number of successes: 7520/10000
Average number of steps: 74.48404255319149
Converged at 9-th iteration.
Number of successes: 7573/10000
Average number of steps: 74.69694968968705

Number of successes of Value Iteration in FrozenLake8x8-v1 : 7520/10000,  Time find the optimal policy : 0.6756384372711182s
Number of successes of Policy Iteration in FrozenLake8x8-v1 : 7573/10000, Time find the optimal policy : 0.9585039615631104s

--------------------------------------------------

Iter  2
Converged at 117-th iteration.
Number of successes: 7533/10000
Average number of steps: 74.54042214257268
Converged at 9-th iteration.
Number of successes: 7422/10000
Average number of steps: 74.79884128267314

Number of successes of Value Iteration in FrozenLake8x8-v1 : 7533/10000,  Time find the optimal policy : 0.7504284381866455s
Number of successes of Policy Iteration in FrozenLake8x8-v1 : 7422/10000, Time find the optimal policy : 0.98189377784729s

----------

Taxi-v3

In [41]:
env = gym.make('Taxi-v3')

In [42]:
vi_iter_avg = 0
pi_iter_avg = 0

vi_rate_avg = 0
pi_rate_avg = 0

vi_step_avg = 0
pi_step_avg = 0

vi_time_avg = 0
pi_time_avg = 0

vi_find_optimal_avg = 0
pi_find_optimal_avg = 0

for i in range(0, 3):
    print('Iter ', i + 1)
    start_vi = time.time()
    vi_value,vi_iter = value_iteration(env, MAX_ITERS, GAMMA)
    vi_time = time.time() - start_vi
    policy_from_value = policy_extraction(env, vi_value, GAMMA)
    start_vi_avg = time.time()
    vi_successes, avg_vi_steps = play_multiple_times(env, policy_from_value, MAX_EPISODES)
    vi_avg = time.time() - start_vi_avg
    start_pi = time.time()
    pi, pi_iter = policy_iteration(env, MAX_ITERS, gamma=GAMMA)
    pi_time = time.time() - start_pi
    start_pi_avg = time.time()
    pi_successes, avg_pi_steps = play_multiple_times(env, pi, MAX_EPISODES)
    pi_avg = time.time() - start_pi_avg
    print()
    print(f'Number of successes of Value Iteration in Taxi-v3 : {vi_successes}/{MAX_EPISODES},  Time find the optimal policy : {vi_time}s')
    print(f'Number of successes of Policy Iteration in Taxi-v3 : {pi_successes}/{MAX_EPISODES}, Time find the optimal policy : {pi_time}s')
    print()
    print('-' * 50)
    print()
    vi_iter_avg += vi_iter/3
    pi_iter_avg += pi_iter/3

    vi_rate_avg += float(vi_successes)/MAX_EPISODES/3
    pi_rate_avg += float(pi_successes)/MAX_EPISODES/3

    vi_step_avg += avg_vi_steps/3
    pi_step_avg += avg_pi_steps/3

    vi_time_avg += vi_avg/MAX_EPISODES/3
    pi_time_avg += pi_avg/MAX_EPISODES/3

    vi_find_optimal_avg += vi_time/3
    pi_find_optimal_avg += pi_time/3

statistics.append(['Taxi-v3', vi_iter_avg, pi_iter_avg, vi_rate_avg, pi_rate_avg, vi_step_avg, pi_step_avg, vi_time_avg, pi_time_avg, vi_find_optimal_avg, pi_find_optimal_avg])

Iter  1
Converged at 116-th iteration.
Number of successes: 10000/10000
Average number of steps: 13.0825
Converged at 16-th iteration.
Number of successes: 10000/10000
Average number of steps: 13.0238

Number of successes of Value Iteration in Taxi-v3 : 10000/10000,  Time find the optimal policy : 6.195146322250366s
Number of successes of Policy Iteration in Taxi-v3 : 10000/10000, Time find the optimal policy : 15.757557392120361s

--------------------------------------------------

Iter  2
Converged at 116-th iteration.
Number of successes: 10000/10000
Average number of steps: 13.0423
Converged at 16-th iteration.
Number of successes: 10000/10000
Average number of steps: 13.0717

Number of successes of Value Iteration in Taxi-v3 : 10000/10000,  Time find the optimal policy : 5.707596778869629s
Number of successes of Policy Iteration in Taxi-v3 : 10000/10000, Time find the optimal policy : 15.352566957473755s

--------------------------------------------------

Iter  3
Converged at 116

##Comparision

In [43]:
df = pd.DataFrame(statistics, columns=['Environment', 'VI Iteration', 'PI Iteration', 'VI Success Rate', 'PI Success Rate', 'Average VI Steps', 'Average PI Steps', 'VI Time', 'PI Time', 'Avg VI Find Optimal Time', 'Avg PI Find Optimal Time'])
df

,Environment,VI Iteration,PI Iteration,VI Success Rate,PI Success Rate,Average VI Steps,Average PI Steps,VI Time,PI Time,Avg VI Find Optimal Time,Avg PI Find Optimal Time
0,FrozenLake-v1,79.0,5.0,0.781367,0.7795,43.596873,43.443798,0.000863,0.000848,0.099310,0.117507
1,FrozenLake8x8-v1,117.0,9.0,0.751167,0.7492,74.516174,74.942522,0.001428,0.001445,0.743186,1.081016
2,Taxi-v3,116.0,16.0,1.000000,1.0000,13.077233,13.047000,0.000480,0.000411,6.254758,15.467462


Based on the experiment by running the code, here are some general observations:

* **Number of iterations to converge**: The Policy
Iteration (PI) algorithm converges faster than the Value Iteration (VI) algorithm across all environments. This is reflected in the number of iterations needed for the algorithm to converge.
* **Success rate**: Both the VI and PI algorithms have a similar success rate across all environments. Notably, both algorithms achieve an absolute success rate (100%) on the Taxi-v3 environment.
* **Average number of steps**: The average number of steps needed to complete an episode is very similar between VI and PI across all environments. This indicates that both algorithms are learning effective strategies.
* **Average time**: The average time to complete an episode is also very similar between VI and PI.
* **Average time find optimal policy**:  The average time to find the optimal policy is higher for PI than for VI in all environments.